## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kralendijk,BQ,12.1500,-68.2667,76.98,83,29,9.22,scattered clouds
1,1,Buchanan,LR,5.8808,-10.0467,77.05,88,89,5.08,light rain
2,2,Bluff,NZ,-46.6000,168.3333,60.80,67,0,13.67,clear sky
3,3,Kapaa,US,22.0752,-159.3190,78.78,77,40,12.66,scattered clouds
4,4,Haines Junction,CA,60.7522,-137.5108,37.35,56,43,13.80,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kralendijk,BQ,12.1500,-68.2667,76.98,83,29,9.22,scattered clouds
1,1,Buchanan,LR,5.8808,-10.0467,77.05,88,89,5.08,light rain
3,3,Kapaa,US,22.0752,-159.3190,78.78,77,40,12.66,scattered clouds
5,5,Port Blair,IN,11.6667,92.7500,82.44,74,40,10.36,scattered clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,82.51,68,26,14.41,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                211
City                   211
Country                210
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_clean_df = preferred_cities_df.dropna()
preferred_clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kralendijk,BQ,12.1500,-68.2667,76.98,83,29,9.22,scattered clouds
1,1,Buchanan,LR,5.8808,-10.0467,77.05,88,89,5.08,light rain
3,3,Kapaa,US,22.0752,-159.3190,78.78,77,40,12.66,scattered clouds
5,5,Port Blair,IN,11.6667,92.7500,82.44,74,40,10.36,scattered clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,82.51,68,26,14.41,scattered clouds


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kralendijk,BQ,76.98,scattered clouds,12.1500,-68.2667,
1,Buchanan,LR,77.05,light rain,5.8808,-10.0467,
3,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,
5,Port Blair,IN,82.44,scattered clouds,11.6667,92.7500,
9,Hithadhoo,MV,82.51,scattered clouds,-0.6000,73.0833,
21,Kavieng,PG,83.73,broken clouds,-2.5744,150.7967,
26,Saint-Philippe,RE,83.79,overcast clouds,-21.3585,55.7679,
27,Hobart,AU,80.02,few clouds,-42.8794,147.3294,
28,Bengkulu,ID,83.26,moderate rain,-3.8004,102.2655,
36,Souillac,MU,79.36,overcast clouds,-20.5167,57.5167,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))